## **Three dims reconstruction of surface mesh**

    This notebook demonstrate:
    1.3D surface mesh reconstruction of the whole body;
    2.3D surface mesh reconstruction of the tissues;
    3.Merge all generated surfaces and save as vtk file;
    4.Read vtk files and visualize.

Although most of the functions of 3D reconstruction can be used in jupyter notebook, I prefer to use an edition like PyCharm.
Most widgets cannot be used in jupyter notebook, but only in vtk rendering window, so PyCharm is highly recommended.

Additionally, The visualization of surface mesh in jupyter notebook is poor due to rendering.
Please copy the following code into an arbitrary `.py` file to run in PyCharm.

### **Packages**

In [ ]:
import anndata as ad
import stDrosophila as sd

### **Example data (Drosophila E16-18-a)**

In [ ]:
file = r"/home/yao/BGIpy37_pytorch113/drosophila_E16_18/E16-18_a_SCT_anno.h5ad"
adata = ad.read(file)
adata.obs["x"] = adata.obs["x"] - adata.obs["x"].min()
adata.obs["y"] = adata.obs["y"] - adata.obs["y"].min()
adata.obs["z"] = adata.obs["z"] - 4.9
adata.obs[["x", "y", "z"]] = adata.obs[["x", "y", "z"]].round(2)
adata.obsm["spatial"] = adata.obs[["x", "y", "z"]].values
print(adata)
print(adata.obs["anno"].unique())

### **3D reconstrcution of the whole body**
3D surface mesh can be reconstructed by this method.

#### `sd.tl.construct_surface`: Construct a surface mesh based on 3D point cloud model.

In [ ]:
pcd = sd.tl.construct_pcd(adata=adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="rainbow", alphamap=1.0)
surf, pcd = sd.tl.construct_surface(pcd=pcd, key_added="groups", color="gainsboro", alpha=0.5, surface_smoothness=200)
sd.pl.three_d_plot(mesh=surf, key="groups", jupyter=True)

### **3D reconstrcution of the tissue**

tissue 1: fat body

In [ ]:
fb_adata = adata[adata.obs["anno"] == "fat body", :].copy()
fb_adata = sd.tl.om_EllipticEnvelope(adata=fb_adata, coordsby="spatial", threshold=0.2)
fb_adata = sd.tl.om_kde(adata=fb_adata, coordsby="spatial", threshold=0.2, kernel="gaussian", bandwidth=1.0)
fb_pcd = sd.tl.construct_pcd(adata=fb_adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="blue", alphamap=1.0)
fb_surf, fb_pcd = sd.tl.construct_surface(pcd=fb_pcd, key_added="groups", color="skyblue", alpha=0.5, surface_smoothness=200)
fb_mesh = sd.tl.collect_mesh([fb_surf, fb_pcd])
sd.pl.three_d_plot(mesh=fb_mesh, key="groups", jupyter=True)

tissue 2: CNS